## Modelagem da especificação 7

In [17]:
#Importando bibliotecas necessarias
import pylab as pb

#Armazenando o preco, custo CC (caminhao) e CF (ferrovia) respectivamente
prices_info = pb.loadtxt('projeto2.txt', dtype = float, max_rows = 3)

#Armazenando o número de fabricas, centros e cidades
instance_info = pb.loadtxt('projeto2.txt', dtype = int, skiprows = 3, max_rows = 3)
print(instance_info[2])

#Armazenando dados associados a cada fabrica, centro e cidade:
dataFab = pb.loadtxt('projeto2.txt', dtype = int, skiprows = 6, max_rows = instance_info[0])

dataCenter = pb.loadtxt('projeto2.txt', dtype = int, skiprows = 6 + instance_info[0], max_rows = instance_info[1])

dataCity = pb.loadtxt('projeto2.txt', dtype = int, skiprows = 6 + instance_info[0] + instance_info[1], 
                      max_rows = instance_info[2])


skip_rows = sum(instance_info) + len(prices_info) + len(instance_info)

#Armazenando os dados dos arcos
dataSet = pb.loadtxt('projeto2.txt',dtype = int, skiprows = skip_rows)


26


In [18]:
#Criando um dicionário que organiza melhor os dados relacionados aos arcos (distancia):
distanceList = {}
for i in range(len(dataSet)):
    distanceList[(dataSet[i][0], dataSet[i][1])] = dataSet[i][2]

In [19]:
print(distanceList)

{(1, 5): 1341, (1, 6): 1432, (1, 7): 1350, (1, 8): 2950, (1, 9): 3242, (1, 10): 1369, (1, 11): 4467, (1, 12): 637, (1, 13): 498, (1, 14): 805, (1, 15): 3707, (1, 16): 1235, (1, 17): 3745, (1, 18): 2385, (1, 19): 3226, (1, 20): 3682, (1, 21): 3720, (1, 22): 2155, (1, 23): 885, (1, 24): 2781, (1, 25): 3433, (1, 26): 3315, (1, 27): 1333, (1, 28): 2746, (1, 29): 3279, (1, 30): 908, (1, 31): 3211, (1, 32): 1786, (1, 33): 1340, (2, 5): 1485, (2, 6): 921, (2, 7): 566, (2, 8): 2182, (2, 9): 2842, (2, 10): 601, (2, 11): 4665, (2, 12): 1012, (2, 13): 476, (2, 14): 773, (2, 15): 3133, (2, 16): 835, (2, 17): 2775, (2, 18): 2805, (2, 19): 2458, (2, 20): 3880, (2, 21): 2952, (2, 22): 1727, (2, 23): 1177, (2, 24): 2979, (2, 25): 2665, (2, 26): 3513, (2, 27): 511, (2, 28): 1982, (2, 29): 2879, (2, 30): 99, (2, 31): 2698, (2, 32): 959, (2, 33): 1484, (3, 5): 2449, (3, 6): 1380, (3, 7): 399, (3, 8): 322, (3, 9): 1984, (3, 10): 1256, (3, 11): 5678, (3, 12): 2537, (3, 13): 2269, (3, 14): 2566, (3, 15): 12

In [20]:
#Criando conjunto de fabricas, centros e cidades
#Criando ranges para facilitar a manipulacao dos dados
fab_r = range(instance_info[0])
cent_r = range(instance_info[1])
city_r = range(instance_info[2])

#Criando conjunto de Fabricas:
factories=[]
for i in fab_r:
    factories.insert(i, dataFab[i][0])

#Criando conjunto de centros:
centers=[]
for i in cent_r:
    centers.insert(i, dataCenter[i][0])

#Criando conjunto de cidades:
cities=[]
for i in city_r:
    cities.insert(i, dataCity[i][0])
    

In [21]:
#Criando unioes entre conjunto de fabricas, centros e cidades:
#Fabrica uniao centros
factoryUcenter = factories + centers 

#Centros uniao cidades
centerUcity = centers + cities

In [22]:
#Criando conjunto com os custos de producao por tonelada Ci de cada fabrica (utilizando um dicionario):
costFactory = {}
for i in fab_r:
    costFactory[(dataFab[i][0])] = dataFab[i][1]
print(costFactory)

#Criando conjunto com as taxas anuais a serem pagas por cada centro de distribuicao utilizado:
anualTax={}
for i in cent_r:
    anualTax[(dataCenter[i][0])] = dataCenter[i][1]
print(anualTax)

#Criando conjunto com as capacidades maximas de producao de cada fabrica (utilizando um dicionario):
capacities={}
for i in fab_r:
    capacities[(dataFab[i][0])] = dataFab[i][2]
print(capacities)

#Criando conjunto com as demandas de cada cidade (utilizando um dicionario):
demands ={}
for i in city_r:
    demands[(dataCity[i][0])] = dataCity[i][1]
print(demands)

{1: 2, 2: 3, 3: 4, 4: 3}
{5: 50, 6: 70, 7: 65}
{1: 2000, 2: 3000, 3: 4500, 4: 3500}
{8: 600, 9: 400, 10: 150, 11: 100, 12: 300, 13: 480, 14: 150, 15: 300, 16: 180, 17: 200, 18: 80, 19: 90, 20: 400, 21: 600, 22: 700, 23: 500, 24: 400, 25: 100, 26: 100, 27: 210, 28: 450, 29: 120, 30: 300, 31: 450, 32: 200, 33: 150}


In [23]:
#Modelagem
from docplex.mp.model import Model
mdl = Model("Problema de especificacao 7")

In [24]:
#Criando variaveis X_i_j
x = {(i, j): mdl.continuous_var(name="x_{0}_{1}".format(i, j)) for (i, j) in distanceList}

mdl.print_information()

Model: Problema de especificacao 7
 - number of variables: 194
   - binary=0, integer=0, continuous=194
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [25]:
#Criando variaveis Y_k, sendo k o centro de distribuicao:
y = {i: mdl.binary_var(name="y_{0}".format(i)) for i in centers}

mdl.print_information()
print(y)

Model: Problema de especificacao 7
 - number of variables: 197
   - binary=3, integer=0, continuous=194
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
{5: docplex.mp.Var(type=B,name='y_5'), 6: docplex.mp.Var(type=B,name='y_6'), 7: docplex.mp.Var(type=B,name='y_7')}


In [26]:
for (i, j) in distanceList:
    mdl.add_constraint(x[i, j] >= 0)

In [27]:
#Adicionando restricao de conservacao de fluxo:
for k in centers:
    mdl.add_constraint(mdl.sum(x[k, j] for j in cities) == mdl.sum(x[i, k] for i in factories))
        

In [28]:
#Adicionando restricao sobre a capacidade produtiva maxima de cada fabrica:
for i in factories:
    mdl.add_constraint(mdl.sum(x[i, j] for j in centerUcity) <= capacities[i])

In [29]:
#Adicionando restricao que limita a quantidade de X_i_j enviada
for j in cities:
    mdl.add_constraint(mdl.sum(x[i, j] for i in factoryUcenter) <= demands[j])

In [30]:
#Adicionando restricao que conserva o fluxo associado a utilizacao ou nao dos centros de distribuicao:
for k in centers:
    mdl.add_constraint(mdl.sum(x[k, j] for j in cities) <= mdl.sum(demands[j]*y[k]))

In [31]:
#Adicionando a funcao objetivo:
mdl.maximize(mdl.sum(prices_info[0]* x[i, j] for j in cities for i in factoryUcenter) - 
             mdl.sum(costFactory[i]*x[i, j] for i in factories for j in centerUcity) - 
             mdl.sum(anualTax[k]*y[k] for k in centers) - 
             mdl.sum(prices_info[1] * x[i, j] * distanceList[i, j] for j in cities for i in factoryUcenter) - 
             mdl.sum(prices_info[2] * x[i, j] * distanceList[i, j] for i in factories for j in centers))

In [32]:
solution = mdl.solve(log_output=True)

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.01 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 194 rows and 0 columns.
Reduced MIP has 36 rows, 197 columns, and 469 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.22 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 36 rows, 197 columns, and 469 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.18 ticks)
Probing time = 0.02 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.01 sec. (0.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   1.6026

In [33]:
solution.display()

solution for: Problema de especificacao 7
objective: 3822446.000
x_1_5 = 150.000
x_1_11 = 100.000
x_1_12 = 300.000
x_1_20 = 400.000
x_1_23 = 500.000
x_1_24 = 400.000
x_1_26 = 100.000
x_2_6 = 150.000
x_2_10 = 150.000
x_2_13 = 480.000
x_2_14 = 150.000
x_2_16 = 30.000
x_2_27 = 210.000
x_2_30 = 300.000
x_2_32 = 200.000
x_3_8 = 600.000
x_3_15 = 300.000
x_3_19 = 90.000
x_3_21 = 600.000
x_3_25 = 100.000
x_3_28 = 450.000
x_4_7 = 150.000
x_4_9 = 400.000
x_4_17 = 50.000
x_4_18 = 80.000
x_4_22 = 700.000
x_4_29 = 120.000
x_4_31 = 450.000
x_5_33 = 150.000
x_6_16 = 150.000
x_7_17 = 150.000
y_5 = 1
y_6 = 1
y_7 = 1
